Build a classification model on train dataset. Make predictions on the test dataset.
Upload your predictions (in .csv file) with your codes as well.


# Importing all important libraries

In [1]:
import nltk #toolkit
import numpy as np # linear algebra
import pandas as pd # data processing, 
import sklearn
import gensim
import multiprocessing
import pandas
import nltk
import turkishnlp
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [2]:
#pip install turkishnlp

In [3]:
#pip install stop-words

In [4]:
from turkishnlp import detector
obj = detector.TurkishNLP()

In [5]:
#obj.download()

In [6]:
#obj.create_word_set()

In [7]:
import locale
locale.setlocale(locale.LC_ALL, 'tr_TR')

'tr_TR'

used to reproduce the same result every time if the script is kept
consistent otherwise each run will produce different results

In [8]:
np.random.seed(500)

# Importing Train Dataset

In [9]:
df = pd.read_csv('train.csv')


In [10]:
df.columns=["Label","feature"]

In [11]:
print(df.head(40))

          Label                                            feature
0   kultursanat  Belgelemek ve değiştirmek için kamera!  4. Han...
1     teknoloji  ﻿  \tE-ticaret alanının yenilikçi bir markası ...
2        saglik  ﻿ RADİKAL -  Sağlık çalışanlarına şiddet uygul...
3   kultursanat  ﻿  \tDeniz Kuvvetleri Komutanlığı bünyesinde S...
4       siyaset  ﻿ RADİKAL -                    CHP            ...
5          spor  Bu kadro başaramaz  'Milli Takım'ın bu yapısıy...
6     teknoloji  ﻿                                      \t     ...
7     teknoloji  ﻿   \t\t\t\t\t\t\t  \t\t\t\t\t\t\t  \t\t\t\t\t...
8       ekonomi  ﻿ RADİKAL -  IMF sözcüsü,                   Yu...
9          spor  ﻿   \t\t\t\t\t\t\t  \t\t\t\t\t\t\t  \t\t\t\t\t...
10      ekonomi  ﻿  \tBorsa İstanbul (BİST) 100 Endeksi günü yü...
11       saglik  ﻿                                      \tTürk ...
12       saglik  ﻿ RADİKAL -  Hastaneye kaldırılan kişi acil ol...
13  kultursanat  Pakistan, Orhan Kemal'in kaleminden Nazım'ı o

In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 2 columns):
Label      2520 non-null object
feature    2520 non-null object
dtypes: object(2)
memory usage: 39.5+ KB
None


In [13]:
print(df.describe(include='all'))

            Label                                            feature
count        2520                                               2520
unique          6                                               2438
top     teknoloji  Ansen sizi sinemanın büyülü dünyasına davet ed...
freq          432                                                  4


In [14]:
df.dropna()

,Label,feature
0,kultursanat,Belgelemek ve değiştirmek için kamera! 4. Han...
1,teknoloji,﻿ \tE-ticaret alanının yenilikçi bir markası ...
2,saglik,﻿ RADİKAL - Sağlık çalışanlarına şiddet uygul...
3,kultursanat,﻿ \tDeniz Kuvvetleri Komutanlığı bünyesinde S...
4,siyaset,﻿ RADİKAL - CHP ...
...,...,...
2515,teknoloji,Artık telefon üretmeyecek ABD merkezli teknol...
2516,spor,﻿ ANKARA ...
2517,spor,﻿ Başantren...
2518,saglik,-50 derecede dondurulan kalbe ritim tedavisi ...


In [15]:
df['feature'] = df['feature'].str.replace('[^a-z|ğ|Ğ|ç|C|ı|I|i|İ|ö|Ö|Ş|ş|ü|Ü| ] ',' ').str.lower()

In [16]:
df['Label'] = df['Label'].str.replace('[^a-z|ğ|Ğ|ç|C|ı|I|i|İ|ö|Ö|Ş|ş|ü|Ü| ] ',' ').str.lower()

In [17]:
print(df.head(50))

          Label                                            feature
0   kultursanat  belgelemek ve değiştirmek için kamera  4 hangi...
1     teknoloji    \te-ticaret alanının yenilikçi bir markası o...
2        saglik   radi̇ka   sağlık çalışanlarına şiddet uygulay...
3   kultursanat    \tdeniz kuvvetleri komutanlığı bünyesinde sa...
4       siyaset   radi̇ka                     ch               ...
5          spor  bu kadro başaramaz  'milli takım'ın bu yapısıy...
6     teknoloji                                        \t      ...
7     teknoloji     \t\t\t\t\t\t  \t\t\t\t\t\t  \t\t\t\t\t\t\t\...
8       ekonomi   radi̇ka   im sözcüsü                   yunani...
9          spor     \t\t\t\t\t\t  \t\t\t\t\t\t  \t\t\t\t\t\t\t\...
10      ekonomi    \tborsa i̇stanbul (bi̇st 10 endeksi günü yüz...
11       saglik                                        \ttürk e...
12       saglik   radi̇ka   hastaneye kaldırılan kişi acil olar...
13  kultursanat  pakistan orhan kemal'in kaleminden nazım'ı ok

In [18]:
len(df)

2520

In [19]:
#df['Label'] = df['Label'].str.split()

In [20]:
#df['feature'] = df['feature'].str.split()

In [21]:
#print(df.head(50))

# Preparaing dataset for model


Prepare Train and Test Data sets
The Corpus will be split into two data sets, Training and Test.
The training data set will be used to fit the model and 
the predictions will be performed on the test data set.
This can be done through the train_test_split from the sklearn library.
The Training Data will have 70% of the corpus and
Test data will have the remaining 30% as we have set the parameter test_size=0.3 .

# 1. TF-IDF Vectorizer

In [22]:
X = df['feature']
y = df['Label']

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
K=1000
print("Performance for the traditional BOW approach with bigrams")
vectorizer = TfidfVectorizer(max_df=0.95, min_df=10, max_features=K, ngram_range=(1,2))
X_train = vectorizer.fit_transform(X)
print("X_train shape: ",X_train.shape)

Performance for the traditional BOW approach with bigrams
X_train shape:  (2520, 1000)


In [24]:
X_train[0].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.2332844 ,
        0.        , 0.        , 0.13408914, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.11931073, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [25]:
list(vectorizer.vocabulary_)[:100]

['ve',
 'için',
 'hangi',
 'film',
 'festivali',
 'yaşanan',
 'hak',
 'ne',
 'kapsamında',
 'aralık',
 '201',
 'türkiye',
 'den',
 'yunanistan',
 'dan',
 'ilişkin',
 'önemli',
 'bir',
 'ev',
 'yapacak',
 'lk',
 'tarihi',
 'benzer',
 'teknik',
 'sinema',
 'ile',
 'son',
 'iki',
 'günü',
 'kadar',
 'gerekiyor',
 'bilgi',
 '2012',
 'yer',
 'galatasaray',
 'ahmet',
 'çelik',
 'film festivali',
 'önemli bir',
 'ticaret',
 'olan',
 'artık',
 'özel',
 'hızlı',
 'ürün',
 'yönetim',
 'satış',
 'ise',
 'ön',
 'düşük',
 'hiçbir',
 'açısından',
 'oldukça',
 'kolay',
 'olacak',
 'aynı',
 'zamanda',
 'yatırım',
 'gerek',
 'hemen',
 'arasında',
 'çok',
 'küçük',
 'orta',
 'büyük',
 'iş',
 'yeni',
 'dönem',
 'de',
 'edecek',
 'in',
 'mustafa',
 'devam',
 'haline',
 'dedi',
 'öne',
 'çalışma',
 'durumu',
 'bağlı',
 'ileri',
 'zaman',
 'gibi',
 'farklı',
 'da',
 'mümkün',
 'bunların',
 'dışında',
 'hizmet',
 'göre',
 'olarak',
 'verdiği',
 'yaparak',
 'çıkan',
 'sağlıklı',
 'özelliği',
 'gıda',
 'çeşitl

In [26]:
from sklearn.model_selection import train_test_split



In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

using 30 % of data for testing and 70% for training

In [28]:
len(X_train)

1764

In [29]:
len(X_test)


756

In [30]:
X_train

2469    piyasaların obama kutlaması almanya ile kabusa...
1549     radi̇ka   yüksek seçim kurulu (ysk başkanı sa...
1456    ﻿ülkede kanser hastalığı üzerine araştırmaları...
1168    şeker hastaları kampa girdi  diyabet konusunda...
691       \tyeşil-beyazlı kulüpten yapılan açıklamada ...
                              ...                        
845     bu besinler göğüsleri büyütüyor  çevrenizde ye...
584     ﻿madri̇  portekiz basını                      ...
287     alex için dev organizasyon  fenerbahçe ile yol...
273       \tküçükçekmece belediyesi halkalı kültür ve ...
951     doğan holding'in kurumsal yönetim notu 1 üzeri...
Name: feature, Length: 1764, dtype: object

In [31]:
X_test

4        radi̇ka                     ch               ...
646       \ttürkiye profesyonel futbolcular derneği ba...
288     ﻿müni̇  brezilyalı uzmanlar anne sütü ile besl...
2212    başkent blues ustalarını ağırlayacak  "efes pi...
1274                                          \t     \...
                              ...                        
1480     radi̇ka -hazine müsteşarlığının sigorta acent...
52      ﻿nyo  i̇sviçre'nin nyon kentindeki uef merkezi...
1724      \tcumhurbaşkanı recep tayyip erdoğan grand t...
294                                           \tapple’...
511     anadilde savunma komisyonda görüşülecek  mecli...
Name: feature, Length: 756, dtype: object

In [32]:
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import sklearn

In [34]:
print(X_train)

2469    piyasaların obama kutlaması almanya ile kabusa...
1549     radi̇ka   yüksek seçim kurulu (ysk başkanı sa...
1456    ﻿ülkede kanser hastalığı üzerine araştırmaları...
1168    şeker hastaları kampa girdi  diyabet konusunda...
691       \tyeşil-beyazlı kulüpten yapılan açıklamada ...
                              ...                        
845     bu besinler göğüsleri büyütüyor  çevrenizde ye...
584     ﻿madri̇  portekiz basını                      ...
287     alex için dev organizasyon  fenerbahçe ile yol...
273       \tküçükçekmece belediyesi halkalı kültür ve ...
951     doğan holding'in kurumsal yönetim notu 1 üzeri...
Name: feature, Length: 1764, dtype: object


# Checking train Accuracy

In [35]:
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import sklearn

# Naive Bayes Accuracy

In [39]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y)
     predict the labels on validation dataset
     predictions_NB = Naive.predict(Test_X_Tfidf)
         Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

IndentationError: unexpected indent (<ipython-input-39-929a3e4a4414>, line 4)

# SVM Accuracy

In [37]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

NameError: name 'Train_X_Tfidf' is not defined

# Predicting with Test

In [ ]:
testo = pd.read_csv('test.csv')



In [ ]:
testo.columns=["feature"]

In [ ]:
testo.head(40)

In [ ]:
testo['feature'] = testo['feature'].str.replace('\t', '')

In [ ]:
testo['feature'] = testo['feature'].str.replace('[^a-z|ğ|Ğ|ç|C|ı|I|i|İ|ö|Ö|Ş|ş|ü|Ü| ] ',' ').str.lower()

In [ ]:
testo.head(10)

In [ ]:
X_test = testo['feature'].values
X_test = vectorizer.transform(X_test)

In [ ]:
predictions = Naive.predict(X_test)

In [ ]:
print(X_test.shape)

In [ ]:
list(vectorizer.vocabulary_)[:100]

# Label Prediction with Test

In [ ]:
Naive.predict(Test_y_Tfidf)

In [ ]:
SVM.predict(Test_y_Tfidf)